In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import os, random, sys, time, re
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from torch.nn.utils.rnn import pad_sequence

from sklearn.model_selection import StratifiedKFold, KFold
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from transformers import DistilBertModel, DistilBertConfig,DistilBertTokenizer,BertModel,BertTokenizer,RobertaTokenizer,RobertaModel
from transformers import AutoModel, BertTokenizerFast

In [ ]:
train_csv = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
test_csv = pd.read_csv("../input/commonlitreadabilityprize/test.csv")

subm = pd.read_csv("../input/commonlitreadabilityprize/sample_submission.csv")

y = (train_csv.target.values > 0).astype(int)

In [ ]:
print(train_csv["target"].min())
print(train_csv["target"].max())

In [ ]:
train_csv.shape

In [ ]:
tokenizer = BertTokenizer.from_pretrained('../input/bert-base-uncased',do_lower_case=True)
tokenizer("hello how are you")

In [ ]:
# from nltk.stem import WordNetLemmatizer
# lem=WordNetLemmatizer()
# train_csv.excerpt=train_csv.excerpt.apply(lambda x: lem.lemmatize(x))

In [ ]:
max_length=max(train_csv.excerpt.apply(lambda x: len(tokenizer(x)['input_ids'])))

In [ ]:
class LitDataSet(Dataset):
    def __init__(self, review,tokenizer,max_len,target=None,is_test=False):
        self.review = review
        self.target = target
        self.is_test = is_test
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.review)
    
    def __getitem__(self, idx):
        review = str(self.review[idx])
        review = ' '.join(review.split())
        global inputs
        
        inputs = self.tokenizer.encode_plus(
            review,
            None,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True
        )        
        ids = torch.tensor(inputs['input_ids'], dtype=torch.long)
        mask = torch.tensor(inputs['attention_mask'], dtype=torch.long)
#         token_type_ids = torch.tensor(inputs['token_type_ids'], dtype=torch.long)
        
        if self.is_test:
            return (ids,mask,test_csv['id'][idx])
                    #token_type_ids)
                 
            
        else:    
            targets = torch.tensor(self.target[idx], dtype=torch.float)
            return (ids,mask,targets)
#                   token_type_ids)

In [ ]:
traindata = LitDataSet(train_csv['excerpt'].values,tokenizer,max_length,train_csv['target'].values,is_test=False)
testdata = LitDataSet(test_csv['excerpt'].values,tokenizer,max_length,target=None,is_test=True)

train_loader = DataLoader(traindata, batch_size=4,
                       shuffle=True, num_workers=4)
test_loader = DataLoader(testdata, batch_size=1,
                       shuffle=False, num_workers=4)

In [ ]:
# x,y,z=next(iter(train_loader))

In [ ]:
# model = RobertaModel.from_pretrained('../input/roberta-base')

In [ ]:
# model(x,y)

In [ ]:
class Bert(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = AutoModel.from_pretrained('../input/bert-base-uncased')
#         self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(768, 1)
    
    def forward(self, idss, mask):
        _,out = self.bert(idss,mask,return_dict=False)
#         output = self.dropout(output)
        out = self.fc(out)
        return out



In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = Bert().to(device)

In [ ]:
model=torch.load("../input/commonlit-pytorch-bert/Robert_trained.pth").to(device)

In [ ]:
loss_fn=nn.MSELoss()
# optimizer=optim.AdamW(model.parameters(), 0.0001,\
#                     betas=(0.9, 0.999), weight_decay=1e-1)

In [ ]:
# param_optimizer = list(model.named_parameters())
# no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
# optimizer_parameters = [
#     {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
#      'weight_decay': 0.0001},
#     {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
#      'weight_decay': 0.0}
#     ]  
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

In [ ]:
from sklearn.model_selection import KFold
kfold=KFold(3,shuffle=True)

In [ ]:
epochs=4
for fold, (train_ids, val_ids) in enumerate(kfold.split(traindata)):
    
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)
    print(f'FOLD {fold+1}')
    print('--------------------------------')
    t_loader = DataLoader(traindata, batch_size=16,\
                       num_workers=8,sampler=train_subsampler)
    val_loader = DataLoader(traindata, batch_size=16,
                    num_workers=8,sampler=val_subsampler)

    for epoch in range(1, epochs+1):      
                start_time = time.time()
                tloss = [] 
                vloss=[]
                model.train()

                for texts, attns,target in tqdm(t_loader):
                        texts, attns,target=texts.to(device), attns.to(device), target.to(device)
                        optimizer.zero_grad()
                        outputs = model(texts,attns)
                        loss = loss_fn(outputs.view(-1).float(), target.view(-1).float()).float()
                        tloss.append(loss.item())
                        loss.backward()
                        optimizer.step()
        #                 scheduler.step()
                tloss = np.array(tloss).mean()
                del loss, outputs
                model.eval()
                with torch.no_grad():
                    for texts, attns,target in (val_loader):
                            texts, attns,target=texts.to(device), attns.to(device), target.to(device)
                            outputs = model(texts,attns)
                            loss = loss_fn(outputs.view(-1).float(), target.view(-1).float()).float()
                            vloss.append(loss.item())

                    vloss = np.array(vloss).mean()

                print(f"Epoch....:{epoch} t_oss...:{tloss}...vlosss:{vloss}")
                del loss, outputs
            


In [ ]:
# epochs=7
# from tqdm import tqdm
# for epoch in range(1, epochs+1):      
#             start_time = time.time()
#             tloss = []          
#             model.train()
    
#             for texts, attns,target in tqdm(train_loader):
#                 texts, attns,target=texts.to(device), attns.to(device), target.to(device)
#                 optimizer.zero_grad()
#                 outputs = model(texts,attns)
#                 loss = loss_fn(outputs.view(-1).float(), target.view(-1).float()).float()
#                 tloss.append(loss.item())
#                 loss.backward()
#                 optimizer.step()
# #                 scheduler.step()
#             tloss = np.array(tloss).mean()
# #             vloss = validation_fn(model, vloader, loss_fn)
#             tmetric = tloss**.5
# #             vmetric = vloss**.5
# #             print(raw_line.format(epoch,tloss,tmetric,vmetric,(time.time()-start_time)/60**1))
#             print(f"Epoch....:{epoch} tloss...:{tloss}...tmetric:{tmetric}")
#             del loss, outputs
            

In [ ]:
torch.save(model,"bert_trained.pth")

In [ ]:
model.eval()
preds={}
for texts,attns,idx in tqdm(test_loader):
                texts, attns=texts.to(device), attns.to(device)
                outputs = model(texts,attns)
                preds[idx[0]]=outputs.item()

In [ ]:
preds

In [ ]:
sub=pd.DataFrame({"id":preds.keys(),"target":preds.values()})
sub.to_csv("submission.csv",index=False)
sub.head()

In [ ]:
sub.target.max()